In [1]:
# !pip install mediapipe

#### 선언부분

In [5]:
import cv2
import mediapipe as mp
import numpy as np

max_num_hands = 2 # 두개 손
gesture = {
    0 :'right1', # 추워요 오른손
    1 :'left1',  # 추워요 왼손
    2 : 'right2', # 감사해요 오른손
    3 :'left2',  # 감사해요 왼손
}
Text = []

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=max_num_hands,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# Gesture recognition model
file = np.genfromtxt('word.csv', delimiter=',')
angle = file[:,:-1].astype(np.float32)
label = file[:, -1].astype(np.float32)

knn = cv2.ml.KNearest_create()
knn.train(angle, cv2.ml.ROW_SAMPLE, label)

True

In [6]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        continue

    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    result = hands.process(img)

    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        rps_result = []

        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 3))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z]

            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree
            
            # Inference gesture
            data = np.array([angle], dtype=np.float32)
            
            #angle_data = np.append(angle_data, data, axis=0)
            
            ret, results, neighbours, dist = knn.findNearest(data, 3)
            idx = int(results[0][0])

            # Draw gesture result 제스쳐 값
            if idx in gesture.keys():
                org = (int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0]))
                cv2.putText(img, text=gesture[idx].upper(), org=(org[0], org[1] + 20), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

                rps_result.append({
                    'rps': gesture[idx],
                    'org': org
                })

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            # means
            if len(rps_result) >= 2:
                text = ''

                if rps_result[0]['rps']=='right1':
                    if rps_result[1]['rps']=='left1'     : text = 'cold'
                elif rps_result[0]['rps']=='left1':
                    if rps_result[1]['rps']=='right1'     : text = 'cold' 
                        
                # thanks
                elif rps_result[0]['rps']=='right2':
                    if rps_result[1]['rps']=='left2'     : text = 'thanks'
                elif rps_result[0]['rps']=='left2':
                    if rps_result[1]['rps']=='right2'     : text = 'thanks' 
                    
                cv2.putText(img, text=text,
                            org=(int(img.shape[1] / 2), 100),
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                            fontScale=2,
                            color=(0, 0, 255),
                            thickness=3)
                Text.append(text)

    cv2.imshow('print_word', img)
    if cv2.waitKey(1) == ord('q'):
        break

#### 텍스트로 받기

In [8]:
txtSet = set(Text)
txtlist = list(txtSet)
txtlist

['', 'cold', 'thanks']

In [9]:
cv2.destroyAllWindows()
cap.release()